In [1]:
import json
import yaml
import requests
import folium
from pymongo import MongoClient
import logging
logging.basicConfig(filename='log.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


import random

## Temperatura

In [2]:
API_key =  config['API_key']
lat = 19.295867896783285
lon = -97.76888973800806
url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric'

In [3]:
response = requests.get(url)
data_weather = response.json()
data_weather

{'coord': {'lon': -97.7689, 'lat': 19.2959},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 25.93,
  'feels_like': 24.79,
  'temp_min': 25.93,
  'temp_max': 25.93,
  'pressure': 1010,
  'humidity': 8,
  'sea_level': 1010,
  'grnd_level': 768},
 'visibility': 10000,
 'wind': {'speed': 0.33, 'deg': 97, 'gust': 2.06},
 'clouds': {'all': 0},
 'dt': 1708639647,
 'sys': {'country': 'MX', 'sunrise': 1708606548, 'sunset': 1708648427},
 'timezone': -21600,
 'id': 3530033,
 'name': 'Cuapiaxtla',
 'cod': 200}

In [4]:
weather_description = data_weather['weather'][0]['description']
temp = data_weather['main']['temp']
text_pupup = f"Clima: {weather_description} <br>Temperatura: {temp} °C"

In [5]:
mapa = folium.Map(location=(lat, lon))
folium.Marker(
    location=[lat, lon],
    tooltip= 'Click Here',
    popup  = text_pupup,
    icon  =folium.Icon('cloud')
).add_to(mapa)

mapa

/tmp/ipykernel_163416/168895962.py:6: UserWarning: color argument of Icon should be one of: {'darkred', 'darkpurple', 'darkgreen', 'darkblue', 'beige', 'lightgray', 'lightred', 'orange', 'gray', 'green', 'white', 'black', 'lightblue', 'red', 'purple', 'lightgreen', 'blue', 'cadetblue', 'pink'}.
  icon  =folium.Icon('cloud')


In [6]:
mapa

## Ejemplo con 3 ciudades

In [7]:
mapa_invernaderos = folium.Map(location=(lat, lon))


In [8]:
cities = config['cities']
for city, coord in cities.items():
    lat = coord[0]
    lon = coord[1]

    url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric'
    response = requests.get(url)
    data_weather = response.json()
    weather_description = data_weather['weather'][0]['description']
    temp = data_weather['main']['temp']
    name = data_weather['name']
    text_pupup = f"El clima en {name} es: <br> Clima: {weather_description} <br>Temperatura: {temp} °C"

    folium.Marker(
        location=[lat, lon],
        tooltip= city,
        popup  = text_pupup,
        icon  =folium.Icon(color="blue", icon="info")
    ).add_to(mapa_invernaderos)

In [9]:
mapa_invernaderos

### Almacenamiento en BD

In [10]:
def coneccionDB(mongo_uri:str,database_name:str,collection_name:str):
    "coneccion a una colecccion en base de datos"
    try:
        client = MongoClient(mongo_uri)
        db = client[database_name]
        collection = db[collection_name]
        print(f'coneccion exitosa a la coleccion: {database_name}.{collection_name}')
        return collection 
    except Exception as e:
        logging.warning(e)

In [11]:
API_key =  config['API_key']

_id = random.randint(1,3)
list_cities = list(config['cities'].keys())
cities = config['cities']
code_name = list_cities[_id]
city = cities[list_cities[_id]]
lat = city[0]
lon = city[1]

url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric'

response = requests.get(url)
data_weather = response.json()
data_weather['code_name']= code_name
weather_database = coneccionDB(config['connection_url'], config['db_ecoterra'], config['weather_collection'])
weather_database.insert_one(data_weather)

coneccion exitosa a la coleccion: Ecoterra.weather


InsertOneResult(ObjectId('65d7c5a33d96738dcdde5ec7'), acknowledged=True)

## Forecast

In [12]:
# api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
API_key =  config['API_key']
lat = 19.295867896783285
lon = -97.76888973800806

url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric&lang='fr'"
response = requests.get(url)
data_weather = response.json()

## alerts

In [13]:
temp = data_weather['main']['temp']
temp_min = data_weather['main']['temp_min']
humidity = data_weather['main']['humidity']
pressure = data_weather['main']['pressure']

In [14]:

def check_temperature(temp:float)->str:
    if config['notifications']['temp_min'] <= temp :
        if temp < config['notifications']['temp_max']:
            return 'Normal'
        else:
            return 'Temperatura elevada'
    else:
        return 'Baja temperatura'

def check_pressure(pressure:float)->str:
    if config['notifications']['pressure_min'] <= pressure :
        if pressure < config['notifications']['pressure_max']:
            return 'Normal'
        else:
            return 'Presion atmosferica elevada'
    else:
        return 'Baja presion atmosferica'

def check_humidity(humidity:float)->str:
    if config['notifications']['humidity_min'] <= humidity :
        if humidity < config['notifications']['humidity_max']:
            return 'Normal'
        else:
            return 'Humedad elevada'
    else:
        return 'Baja humedad'



##  Air Pollution 


### Concentración de contaminantes en el aire

La unidad de medida es $\mu g /m^3$


|Qualitative name   |Index  |$SO_2 $    |$NO_2$        |$PM_{10}$       |$PM_{10.5}$   |$O_3$       |$CO$             |
|-------------------|-------|-----------|---------------|---------------|-----------|---------------|-----------------|
|Good 	            |1 	    |[0; 20) 	|[0; 40) 	    |[0; 20) 	    |[0; 10) 	|[0; 60) 	    |[0; 4400)        |
|Fair 	            |2 	    |[20; 80) 	|[40; 70) 	    |[20; 50) 	    |[10; 25) 	|[60; 100) 	    |[4400; 9400)     |
|Moderate 	        |3 	    |[80; 250) 	|[70; 150) 	    |[50; 100) 	    |[25; 50) 	|[100; 140) 	|[9400-12400)     |
|Poor 	            |4 	    |[250; 350) |[150; 200) 	|[100; 200) 	|[50; 75) 	|[140; 180) 	|[12400; 15400)   |
|Very Poor 	        |5 	    |⩾350 	    |⩾200 	       |⩾200 	       | ⩾75 	  |⩾180 	       |⩾15400          |



fuente: [Open Weather Map](https://openweathermap.org/api/air-pollution)

In [23]:
AirPollutionStandars =  config['notifications']['AirPollution']

In [16]:
API_key =  config['API_key']
lat = 19.295867896783285
lon = -97.76888973800806
url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_key}'
response = requests.get(url)
data_weather = response.json()

In [17]:
def GetPollution(coord:list):
    """Coordinates coord= [lat, lon], where lat, lon is latitude and longitude in type float. \n
    Example: [19.29586, -97.768889]. \n
    Invalid formats: 18°55'22.8\" N 98°23'44.3\"W"""

    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_key}&units=metric'
    response = requests.get(url)
    data_pollution = response.json()
    return data_pollution   

Example

In [24]:
data_pollution = GetPollution(config['cities']['Atlixco'])
data_pollution

{'coord': {'lon': -97.7689, 'lat': 19.2959},
 'list': [{'main': {'aqi': 3},
   'components': {'co': 198.6,
    'no': 0.16,
    'no2': 0.98,
    'o3': 120.16,
    'so2': 5.25,
    'pm2_5': 6.92,
    'pm10': 7.31,
    'nh3': 0.16},
   'dt': 1708640055}]}

### functions

In [19]:
so2_lvl  = data_pollution['list'][0]['components']['so2']
no2_lvl  = data_pollution['list'][0]['components']['no2']
pm10_lvl = data_pollution['list'][0]['components']['pm10']
pm2_5_lvl = data_pollution['list'][0]['components']['pm2_5']
o3_lvl   = data_pollution['list'][0]['components']['o3']
co_lvl   = data_pollution['list'][0]['components']['co']

In [20]:
def verification(diccionario, valor):

    for calidad, rango in diccionario.items():
        if 'min' in rango and 'max' in rango:
            if rango['min'] <= valor < rango['max']:
                return calidad
        elif 'min' in rango and valor >= rango['min']:
            return calidad

    return None

def IndexQuality(diccionario:dict, valor:float):
    Index= {'Good':1, 'Fair':2, 'Moderate':3, 'Poor': 4, 'Very Poor':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return (ver_txt, Index[ver_txt])

In [21]:
def check_so2(so2_lvl:float):
    val = IndexQuality(AirPollutionStandars['so2'], so2_lvl)

    return {
        'element': 'so2',
        'value': so2_lvl,
        'status': val[0],
        'index':  val[1]
    }

def check_no2(no2_lvl:float):
    val = IndexQuality(AirPollutionStandars['no2'], no2_lvl)

    return {
        'element': 'no2',
        'value': no2_lvl,
        'status': val[0],
        'index':  val[1]
    }

def check_pm10(pm10_lvl:float):
    val = IndexQuality(AirPollutionStandars['pm10'], pm10_lvl)

    return {
        'element': 'pm10',
        'value': pm10_lvl,
        'status': val[0],
        'index':  val[1]
    }

def check_pm2_5(pm2_5_lvl:float):
    val = IndexQuality(AirPollutionStandars['pm2_5'], pm2_5_lvl)

    return {
        'element': 'pm2_5',
        'value': pm2_5_lvl,
        'status': val[0],
        'index':  val[1]
    }

def check_o3(o3_lvl:float):
    val = IndexQuality(AirPollutionStandars['o3'], o3_lvl)

    return {
        'element': 'o3',
        'value': o3_lvl,
        'status': val[0],
        'index':  val[1]
    }

def check_co(co_lvl:float):
    val = IndexQuality(AirPollutionStandars['co'], co_lvl)

    return {
        'element': 'co',
        'value': co_lvl,
        'status': val[0],
        'index':  val[1]
    }


print(check_no2(no2_lvl))
print(check_so2(so2_lvl))
print(check_pm10(pm10_lvl))
print(check_pm2_5(pm2_5_lvl))
print(check_o3(o3_lvl))
print(check_co(co_lvl))

{'element': 'no2', 'value': 0.98, 'status': 'Good', 'index': 1}
{'element': 'so2', 'value': 5.25, 'status': 'Good', 'index': 1}
{'element': 'pm10', 'value': 7.31, 'status': 'Good', 'index': 1}
{'element': 'pm2_5', 'value': 6.92, 'status': 'Good', 'index': 1}
{'element': 'o3', 'value': 120.16, 'status': 'Moderate', 'index': 3}
{'element': 'co', 'value': 198.6, 'status': 'Good', 'index': 1}


In [22]:
data_pollution

{'coord': {'lon': -97.7689, 'lat': 19.2959},
 'list': [{'main': {'aqi': 3},
   'components': {'co': 198.6,
    'no': 0.16,
    'no2': 0.98,
    'o3': 120.16,
    'so2': 5.25,
    'pm2_5': 6.92,
    'pm10': 7.31,
    'nh3': 0.16},
   'dt': 1708639653}]}